In [1]:
import tensorflow as tf
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

E0000 00:00:1754413111.187783      10 common_lib.cc:612] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:230
I0000 00:00:1754413131.243472      10 service.cc:148] XLA service 0x5a8e19d676b0 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754413131.243509      10 service.cc:156]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1754413131.243513      10 service.cc:156]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1754413131.243522      10 service.cc:156]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1754413131.243525      10 service.cc:156]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1754413131.243528      10 service.cc:156]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1754413131.243530      10 service.cc:156]   Stream

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Number of devices: 1


In [2]:
! pip install -q -U segmentation-models huggingface_hub git+https://github.com/qubvel/classification_models.git git+https://github.com/huggingface/transformers scikit-image tensorflow

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.19.0 which is incompatible.
tensorflow-tpu 2.18.0 requires ml-dtypes<0.5.0,>=0.4.0, but you have ml-dtypes 0.5.3 which is incompatible.
tensorflow-tpu 2.18.0 requires tensorboard<2.19,>=2.18, but you have tensorboard 2.19.0 which is incompatible.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.19.0 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
import warnings

warnings.filterwarnings("ignore")

os.environ["SM_FRAMEWORK"] = "tf.keras"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm
from glob import glob
import cv2
import re
import albumentations as A
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import segmentation_models as sm
from classification_models.keras import Classifiers
from collections import defaultdict

import tensorflow as tf
from skimage import io
from skimage.transform import resize
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import Precision, Recall, F1Score
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.saving import load_model
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.layers import Concatenate, AveragePooling2D, UpSampling2D, Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Lambda, BatchNormalization, Activation, Dropout
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

Segmentation Models: using `tf.keras` framework.


        I'm so tired of love songs, tired of love songs, tired of love songs, tired of love
    Just wanna go home, wanna go home, wanna go home, wanna go homeee
    

In [4]:
classes = pd.read_csv('/kaggle/input/semantic-drone-dataset/class_dict_seg.csv', index_col=False, skipinitialspace=True)
print(classes)
print("\nNumber of classes : ", len(classes.name.unique()))

           name    r    g    b
0     unlabeled    0    0    0
1    paved-area  128   64  128
2          dirt  130   76    0
3         grass    0  102    0
4        gravel  112  103   87
5         water   28   42  168
6         rocks   48   41   30
7          pool    0   50   89
8    vegetation  107  142   35
9          roof   70   70   70
10         wall  102  102  156
11       window  254  228   12
12         door  254  148   12
13        fence  190  153  153
14   fence-pole  153  153  153
15       person  255   22   96
16          dog  102   51    0
17          car    9  143  150
18      bicycle  119   11   32
19         tree   51   51    0
20    bald-tree  190  250  190
21    ar-marker  112  150  146
22     obstacle    2  135  115
23  conflicting  255    0    0

Number of classes :  24


In [5]:
def get_id2label_label2id(df):
    df['index'] = df.index
    id2label = df.set_index('name').to_dict()['index']
    label2id = {v: k for k, v in id2label.items()}
    return id2label, label2id


id2label, label2id = get_id2label_label2id(classes)

print("id2label:", id2label,'-'*30)
print("label2id:", label2id)

id2label: {'unlabeled': 0, 'paved-area': 1, 'dirt': 2, 'grass': 3, 'gravel': 4, 'water': 5, 'rocks': 6, 'pool': 7, 'vegetation': 8, 'roof': 9, 'wall': 10, 'window': 11, 'door': 12, 'fence': 13, 'fence-pole': 14, 'person': 15, 'dog': 16, 'car': 17, 'bicycle': 18, 'tree': 19, 'bald-tree': 20, 'ar-marker': 21, 'obstacle': 22, 'conflicting': 23} ------------------------------
label2id: {0: 'unlabeled', 1: 'paved-area', 2: 'dirt', 3: 'grass', 4: 'gravel', 5: 'water', 6: 'rocks', 7: 'pool', 8: 'vegetation', 9: 'roof', 10: 'wall', 11: 'window', 12: 'door', 13: 'fence', 14: 'fence-pole', 15: 'person', 16: 'dog', 17: 'car', 18: 'bicycle', 19: 'tree', 20: 'bald-tree', 21: 'ar-marker', 22: 'obstacle', 23: 'conflicting'}


In [6]:
H, W = 704, 1056
#H, W = 1024, 1536
shape = (H, W, 3)
num_classes = len(classes.name.unique())
lr = 1e-4
batch_size = 4
epochs = 200

In [7]:
def get_dir_files(directory: str):
    files = os.listdir(directory)
    files.sort(key=lambda x: int(re.search(r'\d+', x).group()) if re.search(r'\d+', x) else 0)
    return [os.path.join(directory, f) for f in files]

In [ ]:
def analyze_class_distribution():
    print("Dataset images")
    classes_distr = defaultdict(list)
    for filepath in tqdm(get_dir_files('/kaggle/input/semantic-drone-dataset/dataset/semantic_drone_dataset/label_images_semantic'), desc="Analyzing class distribution", leave=False):
        labels = np.array(tf.keras.utils.load_img(filepath, color_mode='grayscale'))
        for label, name in classes['name'].to_dict().items():
            classes_distr[name].append((labels == label).mean())
    
    classes_distr = pd.DataFrame(classes_distr)
    stats = pd.DataFrame({
        'mean': classes_distr.mean(),
        'median': classes_distr.quantile(0.5),
        'has_class': (classes_distr > 0).mean()
    }).reset_index().rename(columns={'index': 'class'})
    
    fig = plt.figure(figsize=(15, 5))
    sns.barplot(data=stats.sort_values(['mean'], ascending=False), x='class', y='mean')
    plt.title('Mean of class distribution in image')
    plt.xticks(rotation=45, ha='right')
    plt.show()
    
    fig = plt.figure(figsize=(15, 5))
    sns.barplot(data=stats.sort_values(['median'], ascending=False), x='class', y='median')
    plt.title('Median of class distribution in image')
    plt.xticks(rotation=45, ha='right')
    plt.show()
    
    fig = plt.figure(figsize=(15, 5))
    sns.barplot(data=stats.sort_values(['has_class'], ascending=False), x='class', y='has_class')
    plt.title('"Image has at least one class pixel" distribution')
    plt.xticks(rotation=45, ha='right')
    plt.show()
    
    return stats, classes_distr

stats, class_distribution = analyze_class_distribution()

In [8]:
def create_dir(directory):
    if not os.path.exists(directory): os.makedirs(directory)
    return directory

create_dir('/kaggle/working/logs')

'/kaggle/working/logs'

In [9]:
def build_augmented_dataset(output_dir, images, masks, augment=True):
    
    images_dir, masks_dir = output_dir+'/images', output_dir+'/masks'
    for directory in [output_dir, images_dir, masks_dir]: create_dir(directory)

    for x_path, y_path in tqdm(zip(images, masks), total=len(images)):
        image_base, image_ext = os.path.splitext(os.path.basename(x_path))
        mask_base, mask_ext = os.path.splitext(os.path.basename(y_path))
        image_ext = image_ext.lstrip('.')
        mask_ext = mask_ext.lstrip('.')

        x_img = cv2.imread(x_path, cv2.IMREAD_COLOR)
        y_img = cv2.imread(y_path, cv2.IMREAD_COLOR)
        x_img = cv2.resize(x_img, (W, H))
        y_img = cv2.resize(y_img, (W, H))

        save_images = [x_img]
        save_masks = [y_img]

        if augment:
            crop_aug = A.RandomCrop(int(2*H/3), int(2*W/3), always_apply=True, p=1.0)
            crop_result = crop_aug(image=x_img, mask=y_img)
            x_crop = cv2.resize(crop_result["image"], (W, H))
            y_crop = cv2.resize(crop_result["mask"], (W, H))
            save_images.append(x_crop)
            save_masks.append(y_crop)

            hflip_aug = A.HorizontalFlip(p=1.0)
            hflip_result = hflip_aug(image=x_img, mask=y_img)
            save_images.append(hflip_result["image"])
            save_masks.append(hflip_result["mask"])

            vflip_aug = A.VerticalFlip(p=1.0)
            vflip_result = vflip_aug(image=x_img, mask=y_img)
            save_images.append(vflip_result["image"])
            save_masks.append(vflip_result["mask"])

        for idx, (img, mask) in enumerate(zip(save_images, save_masks)):
            img_path = os.path.join(images_dir, f"{image_base}_{idx}.{image_ext}")
            mask_path = os.path.join(masks_dir, f"{mask_base}_{idx}.{mask_ext}")
            cv2.imwrite(img_path, img)
            cv2.imwrite(mask_path, mask)

    print(f"Data augmentation and saving completed.\nImages: {images_dir}\nMasks: {masks_dir}")
try:
    build_augmented_dataset(
        '/kaggle/working/data', 
        get_dir_files('/kaggle/input/semantic-drone-dataset/dataset/semantic_drone_dataset/original_images'), 
        get_dir_files('/kaggle/input/semantic-drone-dataset/dataset/semantic_drone_dataset/label_images_semantic'),
        augment = True
    )
except KeyboardInterrupt:
    print("Stopped.")

100%|██████████| 400/400 [03:48<00:00,  1.75it/s]


Data augmentation and saving completed.
Images: /kaggle/working/data/images
Masks: /kaggle/working/data/masks


In [10]:
image_path =  '/kaggle/working/data/images'
label_path = '/kaggle/working/data/masks'

#image_path =  '/kaggle/input/semantic-drone-dataset/dataset/semantic_drone_dataset/original_images'
#label_path = '/kaggle/input/semantic-drone-dataset/dataset/semantic_drone_dataset/label_images_semantic'

def create_dataframe(path):
    name = []
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            name.append(filename.split('.')[0])

    return pd.DataFrame({'id': name}, index = np.arange(0, len(name)))

df_images = create_dataframe(image_path)
df_masks = create_dataframe(label_path)
print('Total Images: ', len(df_images))

X_train, X_testval = train_test_split(df_images['id'], test_size=0.25, shuffle=True)
X_val, X_test = train_test_split(X_testval, test_size=0.2, shuffle=True)

print(f"Train Size : {len(X_train)} images")
print(f"Val Size   :  {len(X_val)} images")
print(f"Test Size  :  {len(X_test)} images")

y_train = X_train
y_test = X_test
y_val = X_val

img_train = [os.path.join(image_path, f"{name}.jpg") for name in X_train]
mask_train = [os.path.join(label_path, f"{name}.png") for name in y_train]
img_val = [os.path.join(image_path, f"{name}.jpg") for name in X_val]
mask_val = [os.path.join(label_path, f"{name}.png") for name in y_val]
img_test = [os.path.join(image_path, f"{name}.jpg") for name in X_test]
mask_test = [os.path.join(label_path, f"{name}.png") for name in y_test]

def parse_image(img_path: str, mask_path: str) -> tuple:
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32) / 255.0

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.cast(mask, tf.uint8)

    return img, mask

def preprocess(img: tf.Tensor, mask: tf.Tensor, model_type: str = 'standard') -> tuple:
    img = tf.image.resize(img, (H, W))
    mask = tf.image.resize(mask, (H, W))

    if model_type == 'standard':
        mask = tf.squeeze(mask, axis=-1)
        return img, tf.one_hot(tf.cast(mask, tf.int32), num_classes)

    if model_type == 'segmenter':
        img = tf.transpose(img, perm=(2, 1, 0))
        mask = tf.transpose(mask, perm=(1, 0, 2))
        mask = tf.squeeze(mask, axis=-1)
        return img, mask
    else:
        raise ValueError("Invalid model_type. Choose 'standard' or 'segmenter'.")

def prepare_dataset(images:list, masks:list, model_type: str = 'standard', batch_size:int = batch_size, shuffle: bool = True) -> tf.data.Dataset:

    dataset = tf.data.Dataset.from_tensor_slices((images, masks))

    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(images))

    dataset = dataset.map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.map(lambda x, y: preprocess(x, y, model_type), num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

    return dataset

train_dataset = prepare_dataset(img_train, mask_train, model_type='standard')
val_dataset = prepare_dataset(img_val, mask_val, model_type='standard')

train_dataset_segmenter = prepare_dataset(img_train, mask_train, model_type='segmenter')
val_dataset_segmenter = prepare_dataset(img_val, mask_val, model_type='segmenter')

train_steps = len(img_train)//batch_size
valid_steps = len(img_val)//batch_size

print(f"Training Steps : {train_steps} \nValidation Steps : {valid_steps}")
print(val_dataset.element_spec, train_dataset.element_spec)

Total Images:  1600
Train Size : 1200 images
Val Size   :  320 images
Test Size  :  80 images
Training Steps : 300 
Validation Steps : 80
(TensorSpec(shape=(None, 704, 1056, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 704, 1056, 24), dtype=tf.float32, name=None)) (TensorSpec(shape=(None, 704, 1056, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 704, 1056, 24), dtype=tf.float32, name=None))


In [ ]:
def plot(model):
    return tf.keras.utils.plot_model(
    model,
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir='TB',
    expand_nested=False,
    dpi=46,
    layer_range=None
)

In [11]:
precision = Precision(name='precision')
recall = Recall(name='recall')
mean_iou = sm.metrics.IOUScore()
fscore = sm.metrics.FScore()

CategoricalFocalLoss = sm.losses.CategoricalFocalLoss()
DiceLoss = sm.losses.DiceLoss()


@tf.function
def dice_loss(y_true, y_pred):
    numerator = tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true * y_true) + tf.reduce_sum(y_pred * y_pred) - tf.reduce_sum(y_true * y_pred)
    return 1 - numerator / denominator

In [12]:
def contracting_path(x, filters, name_prefix):
    conv1 = Conv2D(filters, (3, 3), padding='same', name=f'{name_prefix}_conv1')(x)
    conv1 = BatchNormalization(name=f'{name_prefix}_bn1')(conv1)
    conv1 = Activation('relu', name=f'{name_prefix}_relu1')(conv1)
    conv2 = Conv2D(filters, (3, 3), padding='same', name=f'{name_prefix}_conv2')(conv1)
    conv2 = BatchNormalization(name=f'{name_prefix}_bn2')(conv2)
    conv2 = Activation('relu', name=f'{name_prefix}_relu2')(conv2)
    pool = MaxPooling2D((2, 2), name=f'{name_prefix}_pool')(conv2)
    return conv2, pool

def expanding_path(x, skip_connections, filters, name_prefix):
    up = Conv2DTranspose(filters, (2, 2), strides=(2, 2), padding='same', name=f'{name_prefix}_up')(x)
    cat = concatenate([up, skip_connections], name=f'{name_prefix}_concat')
    conv1 = Conv2D(filters, (3, 3), padding='same', name=f'{name_prefix}_conv1')(cat)
    conv1 = BatchNormalization(name=f'{name_prefix}_bn1')(conv1)
    conv1 = Activation('relu', name=f'{name_prefix}_relu1')(conv1)
    conv1 = Dropout(0.5, name=f'{name_prefix}_dropout')(conv1)
    conv2 = Conv2D(filters, (3, 3), padding='same', name=f'{name_prefix}_conv2')(conv1)
    conv2 = BatchNormalization(name=f'{name_prefix}_bn2')(conv2)
    conv2 = Activation('relu', name=f'{name_prefix}_relu2')(conv2)
    return conv2

def build_unet(input_shape=shape, num_classes=num_classes):

    inputs = Input(input_shape, name='input_layer')

    c1, p1 = contracting_path(inputs, 16, name_prefix='block1')
    c2, p2 = contracting_path(p1, 32, name_prefix='block2')
    c3, p3 = contracting_path(p2, 64, name_prefix='block3')
    c4, p4 = contracting_path(p3, 128, name_prefix='block4')

    c5 = Conv2D(256, (3, 3), activation='relu', padding='same', name='bridge_conv1')(p4)
    c5 = Conv2D(256, (3, 3), activation='relu', padding='same', name='bridge_conv2')(c5)

    u4 = expanding_path(c5, c4, 128, name_prefix='up_block4')
    u3 = expanding_path(u4, c3, 64, name_prefix='up_block3')
    u2 = expanding_path(u3, c2, 32, name_prefix='up_block2')
    u1 = expanding_path(u2, c1, 16, name_prefix='up_block1')

    outputs = Conv2D(num_classes, (1, 1), padding='same', activation='softmax', name='output_layer')(u1)

    model = Model(inputs=inputs, outputs=outputs, name='U-Net')

    return model


unet = build_unet()
unet.compile(
    loss = DiceLoss,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[fscore, mean_iou, 'accuracy']
)

try:
    unet.fit(
        train_dataset,
        steps_per_epoch=train_steps,
        validation_data=val_dataset,
        validation_steps=valid_steps,
        batch_size=batch_size,
        epochs=epochs,
    )
except KeyboardInterrupt:
    print(f"{'-'*50}Training Stopped.{ '-'*50}")

Epoch 1/5


I0000 00:00:1753681499.911494     503 service.cc:152] XLA service 0x7d32f4410f10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753681499.911544     503 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753681505.996832     503 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-07-28 05:45:07.978796: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-07-28 05:45:08.176963: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-07-28 05:45:09.387682: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accur

72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0896 - f1-score: 0.0327 - iou_score: 0.0178 - loss: 0.9673

2025-07-28 05:47:22.001720: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-07-28 05:47:22.239623: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-07-28 05:47:22.906991: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-07-28 05:47:23.168121: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


72/72 ━━━━━━━━━━━━━━━━━━━━ 184s 2s/step - accuracy: 0.0910 - f1-score: 0.0328 - iou_score: 0.0179 - loss: 0.9672 - val_accuracy: 0.3443 - val_f1-score: 0.0266 - val_iou_score: 0.0140 - val_loss: 0.9734
Epoch 2/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 109s 2s/step - accuracy: 0.4928 - f1-score: 0.0656 - iou_score: 0.0396 - loss: 0.9344 - val_accuracy: 0.3570 - val_f1-score: 0.0338 - val_iou_score: 0.0218 - val_loss: 0.9662
Epoch 3/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - accuracy: 0.5343 - f1-score: 0.0948 - iou_score: 0.0623 - loss: 0.9052 - val_accuracy: 0.4148 - val_f1-score: 0.0439 - val_iou_score: 0.0284 - val_loss: 0.9561
Epoch 4/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.5850 - f1-score: 0.1184 - iou_score: 0.0812 - loss: 0.8816 - val_accuracy: 0.3935 - val_f1-score: 0.0436 - val_iou_score: 0.0284 - val_loss: 0.9564
Epoch 5/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 109s 2s/step - accuracy: 0.5769 - f1-score: 0.1289 - iou_score: 0.0893 - loss: 0.8711 - val_accuracy: 0.4512 - val_f1-score: 

Epoch 1/5<br>
72/72 ━━━━━━━━━━━━━━━━━━━━ 184s 2s/step - accuracy: 0.0910 - f1-score: 0.0328 - iou_score: 0.0179 - loss: 0.9672 - val_accuracy: 0.3443 - val_f1-score: 0.0266 - val_iou_score: 0.0140 - val_loss: 0.9734<br>
Epoch 2/5<br>
72/72 ━━━━━━━━━━━━━━━━━━━━ 109s 2s/step - accuracy: 0.4928 - f1-score: 0.0656 - iou_score: 0.0396 - loss: 0.9344 - val_accuracy: 0.3570 - val_f1-score: 0.0338 - val_iou_score: 0.0218 - val_loss: 0.9662<br>
Epoch 3/5<br>
72/72 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - accuracy: 0.5343 - f1-score: 0.0948 - iou_score: 0.0623 - loss: 0.9052 - val_accuracy: 0.4148 - val_f1-score: 0.0439 - val_iou_score: 0.0284 - val_loss: 0.9561<br>
Epoch 4/5<br>
72/72 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.5850 - f1-score: 0.1184 - iou_score: 0.0812 - loss: 0.8816 - val_accuracy: 0.3935 - val_f1-score: 0.0436 - val_iou_score: 0.0284 - val_loss: 0.9564<br>
Epoch 5/5<br>
72/72 ━━━━━━━━━━━━━━━━━━━━ 109s 2s/step - accuracy: 0.5769 - f1-score: 0.1289 - iou_score: 0.0893 - loss: 0.8711 - val_accuracy: 0.4512 - val_f1-score: 0.0531 - val_iou_score: 0.0359 - val_loss: 0.9469


In [ ]:
unet_res = sm.Unet(
    backbone_name='resnet34', 
    encoder_weights='imagenet', 
    input_shape=shape, 
    classes=num_classes, 
    activation='softmax'
)

unet_res.compile(
    loss = DiceLoss,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[fscore, mean_iou, 'accuracy']
)

try:
    unet_res.fit(
        train_dataset,
        steps_per_epoch=train_steps,
        validation_data=val_dataset,
        validation_steps=valid_steps,
        batch_size=batch_size,
        epochs=epochs,
    )
except KeyboardInterrupt:
    print(f"{'-'*50}Training Stopped.{ '-'*50}")

85521592/85521592 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/5


2025-07-28 05:59:17.465835: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-07-28 05:59:17.726450: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-07-28 05:59:18.449346: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-07-28 05:59:18.710292: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-07-28 05:59:27.784772: E external/local_xla/xla/stream_

72/72 ━━━━━━━━━━━━━━━━━━━━ 208s 2s/step - accuracy: 0.3628 - f1-score: 0.0720 - iou_score: 0.0452 - loss: 0.9280 - val_accuracy: 0.0028 - val_f1-score: 0.0420 - val_iou_score: 0.0418 - val_loss: 0.9580
Epoch 2/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 115s 2s/step - accuracy: 0.5805 - f1-score: 0.1683 - iou_score: 0.1190 - loss: 0.8317 - val_accuracy: 0.0712 - val_f1-score: 0.0105 - val_iou_score: 0.0056 - val_loss: 0.9895
Epoch 3/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 114s 2s/step - accuracy: 0.6330 - f1-score: 0.2062 - iou_score: 0.1489 - loss: 0.7938 - val_accuracy: 0.3441 - val_f1-score: 0.0325 - val_iou_score: 0.0197 - val_loss: 0.9675
Epoch 4/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 115s 2s/step - accuracy: 0.6509 - f1-score: 0.2349 - iou_score: 0.1725 - loss: 0.7651 - val_accuracy: 0.0046 - val_f1-score: 0.0049 - val_iou_score: 0.0025 - val_loss: 0.9951
Epoch 5/5
43/72 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.6532 - f1-score: 0.2642 - iou_score: 0.1917 - loss: 0.7358

Epoch 1/5<br>
72/72 ━━━━━━━━━━━━━━━━━━━━ 208s 2s/step - accuracy: 0.3628 - f1-score: 0.0720 - iou_score: 0.0452 - loss: 0.9280 - val_accuracy: 0.0028 - val_f1-score: 0.0420 - val_iou_score: 0.0418 - val_loss: 0.9580<br>
Epoch 2/5<br>
72/72 ━━━━━━━━━━━━━━━━━━━━ 115s 2s/step - accuracy: 0.5805 - f1-score: 0.1683 - iou_score: 0.1190 - loss: 0.8317 - val_accuracy: 0.0712 - val_f1-score: 0.0105 - val_iou_score: 0.0056 - val_loss: 0.9895<br>
Epoch 3/5<br>
72/72 ━━━━━━━━━━━━━━━━━━━━ 114s 2s/step - accuracy: 0.6330 - f1-score: 0.2062 - iou_score: 0.1489 - loss: 0.7938 - val_accuracy: 0.3441 - val_f1-score: 0.0325 - val_iou_score: 0.0197 - val_loss: 0.9675<br>
Epoch 4/5<br>
72/72 ━━━━━━━━━━━━━━━━━━━━ 115s 2s/step - accuracy: 0.6509 - f1-score: 0.2349 - iou_score: 0.1725 - loss: 0.7651 - val_accuracy: 0.0046 - val_f1-score: 0.0049 - val_iou_score: 0.0025 - val_loss: 0.9951<br>
Epoch 5/5<br>
43/72 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.6532 - f1-score: 0.2642 - iou_score: 0.1917 - loss: 0.7358

In [10]:
unet_vgg = sm.Unet(
    backbone_name='vgg16', 
    encoder_weights='imagenet', 
    input_shape=shape, 
    classes=num_classes, 
    activation='softmax'
)

unet_vgg.compile(
    loss = DiceLoss,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[fscore, mean_iou, 'accuracy']
)

try:
    unet_vgg.fit(
        train_dataset,
        steps_per_epoch=train_steps,
        validation_data=val_dataset,
        validation_steps=valid_steps,
        batch_size=batch_size,
        epochs=epochs,
    )
except KeyboardInterrupt:
    print(f"{'-'*50}Training Stopped.{ '-'*50}")

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5


I0000 00:00:1753683657.353428     195 service.cc:152] XLA service 0x7db7c000dda0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753683657.353469     195 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753683663.639548     195 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-07-28 06:21:09.745691: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-07-28 06:21:10.069276: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-07-28 06:21:32.665796: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accur

72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4177 - f1-score: 0.0644 - iou_score: 0.0410 - loss: 0.9356

2025-07-28 06:24:47.022676: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-07-28 06:24:47.346453: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


72/72 ━━━━━━━━━━━━━━━━━━━━ 286s 2s/step - accuracy: 0.4187 - f1-score: 0.0648 - iou_score: 0.0413 - loss: 0.9352 - val_accuracy: 0.0749 - val_f1-score: 0.0285 - val_iou_score: 0.0177 - val_loss: 0.9715
Epoch 2/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.5531 - f1-score: 0.1331 - iou_score: 0.0919 - loss: 0.8669 - val_accuracy: 0.1296 - val_f1-score: 0.0252 - val_iou_score: 0.0146 - val_loss: 0.9748
Epoch 3/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.5818 - f1-score: 0.1526 - iou_score: 0.1070 - loss: 0.8474 - val_accuracy: 0.1256 - val_f1-score: 0.0429 - val_iou_score: 0.0256 - val_loss: 0.9571
Epoch 4/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 131s 2s/step - accuracy: 0.5919 - f1-score: 0.1790 - iou_score: 0.1240 - loss: 0.8210 - val_accuracy: 0.0861 - val_f1-score: 0.0306 - val_iou_score: 0.0171 - val_loss: 0.9694
Epoch 5/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 130s 2s/step - accuracy: 0.5962 - f1-score: 0.2040 - iou_score: 0.1407 - loss: 0.7959 - val_accuracy: 0.3741 - val_f1-score: 

Epoch 1/5<br>
72/72 ━━━━━━━━━━━━━━━━━━━━ 286s 2s/step - accuracy: 0.4187 - f1-score: 0.0648 - iou_score: 0.0413 - loss: 0.9352 - val_accuracy: 0.0749 - val_f1-score: 0.0285 - val_iou_score: 0.0177 - val_loss: 0.9715<br>
Epoch 2/5<br>
72/72 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.5531 - f1-score: 0.1331 - iou_score: 0.0919 - loss: 0.8669 - val_accuracy: 0.1296 - val_f1-score: 0.0252 - val_iou_score: 0.0146 - val_loss: 0.9748<br>
Epoch 3/5<br>
72/72 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.5818 - f1-score: 0.1526 - iou_score: 0.1070 - loss: 0.8474 - val_accuracy: 0.1256 - val_f1-score: 0.0429 - val_iou_score: 0.0256 - val_loss: 0.9571<br>
Epoch 4/5<br>
72/72 ━━━━━━━━━━━━━━━━━━━━ 131s 2s/step - accuracy: 0.5919 - f1-score: 0.1790 - iou_score: 0.1240 - loss: 0.8210 - val_accuracy: 0.0861 - val_f1-score: 0.0306 - val_iou_score: 0.0171 - val_loss: 0.9694<br>
Epoch 5/5<br>
72/72 ━━━━━━━━━━━━━━━━━━━━ 130s 2s/step - accuracy: 0.5962 - f1-score: 0.2040 - iou_score: 0.1407 - loss: 0.7959 - val_accuracy: 0.3741 - val_f1-score: 0.1451 - val_iou_score: 0.0935 - val_loss: 0.8549<br>

In [11]:
unet_mobi = sm.Unet(
    backbone_name='mobilenetv2', 
    encoder_weights='imagenet', 
    input_shape=shape, 
    classes=num_classes, 
    activation='softmax'
)

unet_mobi.compile(
    loss = DiceLoss,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[fscore, mean_iou, 'accuracy']
)

try:
    unet_mobi.fit(
        train_dataset,
        steps_per_epoch=train_steps,
        validation_data=val_dataset,
        validation_steps=valid_steps,
        batch_size=batch_size,
        epochs=epochs,
    )
except KeyboardInterrupt:
    print(f"{'-'*50}Training Stopped.{ '-'*50}")

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5


2025-07-28 06:35:21.078535: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-07-28 06:35:21.276243: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-07-28 06:35:49.324002: E external/local_xla/xla/service/slow_operation_alarm.cc:73] Trying algorithm eng0{} for conv %cudnn-conv-bw-filter.124 = (f32[64,272,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[4,272,256,384]{3,2,1,0} %bitcast.66377, f32[4,64,256,384]{3,2,1,0} %bitcast.66379), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", metadata={op_type="Conv2DBackpropFilter" op_name="gradient_tape/functional_1_1/decoder_stage2a_conv_1/convolution/Conv2DBackpropFilter"

72/72 ━━━━━━━━━━━━━━━━━━━━ 230s 2s/step - accuracy: 0.2343 - f1-score: 0.0676 - iou_score: 0.0428 - loss: 0.9324 - val_accuracy: 0.0574 - val_f1-score: 0.0330 - val_iou_score: 0.0191 - val_loss: 0.9670
Epoch 2/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 173s 2s/step - accuracy: 0.5602 - f1-score: 0.1636 - iou_score: 0.1170 - loss: 0.8364 - val_accuracy: 0.0780 - val_f1-score: 0.0323 - val_iou_score: 0.0195 - val_loss: 0.9677
Epoch 3/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 123s 2s/step - accuracy: 0.7235 - f1-score: 0.2511 - iou_score: 0.1869 - loss: 0.7489 - val_accuracy: 0.0971 - val_f1-score: 0.0373 - val_iou_score: 0.0211 - val_loss: 0.9627
Epoch 4/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 152s 2s/step - accuracy: 0.7229 - f1-score: 0.2925 - iou_score: 0.2193 - loss: 0.7075 - val_accuracy: 0.0845 - val_f1-score: 0.0466 - val_iou_score: 0.0275 - val_loss: 0.9534
Epoch 5/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 133s 2s/step - accuracy: 0.7532 - f1-score: 0.3347 - iou_score: 0.2541 - loss: 0.6653 - val_accuracy: 0.0712 - val_f1-score: 

Epoch 1/5<br>
72/72 ━━━━━━━━━━━━━━━━━━━━ 230s 2s/step - accuracy: 0.2343 - f1-score: 0.0676 - iou_score: 0.0428 - loss: 0.9324 - val_accuracy: 0.0574 - val_f1-score: 0.0330 - val_iou_score: 0.0191 - val_loss: 0.9670<br>
Epoch 2/5<br>
72/72 ━━━━━━━━━━━━━━━━━━━━ 173s 2s/step - accuracy: 0.5602 - f1-score: 0.1636 - iou_score: 0.1170 - loss: 0.8364 - val_accuracy: 0.0780 - val_f1-score: 0.0323 - val_iou_score: 0.0195 - val_loss: 0.9677<br>
Epoch 3/5<br>
72/72 ━━━━━━━━━━━━━━━━━━━━ 123s 2s/step - accuracy: 0.7235 - f1-score: 0.2511 - iou_score: 0.1869 - loss: 0.7489 - val_accuracy: 0.0971 - val_f1-score: 0.0373 - val_iou_score: 0.0211 - val_loss: 0.9627<br>
Epoch 4/5<br>
72/72 ━━━━━━━━━━━━━━━━━━━━ 152s 2s/step - accuracy: 0.7229 - f1-score: 0.2925 - iou_score: 0.2193 - loss: 0.7075 - val_accuracy: 0.0845 - val_f1-score: 0.0466 - val_iou_score: 0.0275 - val_loss: 0.9534<br>
Epoch 5/5<br>
72/72 ━━━━━━━━━━━━━━━━━━━━ 133s 2s/step - accuracy: 0.7532 - f1-score: 0.3347 - iou_score: 0.2541 - loss: 0.6653 - val_accuracy: 0.0712 - val_f1-score: 0.0617 - val_iou_score: 0.0360 - val_loss: 0.9383<br>

In [14]:
def convolution_block(x, filters, stage, block):
    conv1 = Conv2D(filters, (3, 3), activation='elu', padding='same', name=f'conv{stage}_{block}_1')(x)
    conv1 = Dropout(0.5, name=f'dropout{stage}_{block}_1')(conv1)
    conv2 = Conv2D(filters, (3, 3), activation='elu', padding='same', name=f'conv{stage}_{block}_2')(conv1)
    conv2 = Dropout(0.5, name=f'dropout{stage}_{block}_2')(conv2)
    pool = MaxPooling2D((2, 2), name=f'pool{stage}_{block}')(conv2)
    return conv2, pool

def upsampling_block(x, filters, stage, block, skip_connections):
    up = Conv2DTranspose(filters, (2, 2), strides=(2, 2), padding='same', name=f'up{stage}_{block}')(x)
    cat = concatenate([up] + skip_connections, name=f'concat{stage}_{block}')
    conv1 = Conv2D(filters, (3, 3), activation='elu', padding='same', name=f'conv{stage}_{block}_1')(cat)
    conv1 = Dropout(0.5, name=f'dropout{stage}_{block}_1')(conv1)
    conv2 = Conv2D(filters, (3, 3), activation='elu', padding='same', name=f'conv{stage}_{block}_2')(conv1)
    conv2 = Dropout(0.5, name=f'dropout{stage}_{block}_2')(conv2)
    return conv2

def build_unet_plusplus(input_shape=shape, num_classes=num_classes):

    inputs = Input(input_shape, name='input')

    c1_1, p1 = convolution_block(inputs, 32, 1, 1)
    c2_1, p2 = convolution_block(p1, 64, 2, 1)
    c3_1, p3 = convolution_block(p2, 128, 3, 1)
    c4_1, p4 = convolution_block(p3, 256, 4, 1)
    c5_1, _ = convolution_block(p4, 512, 5, 1)

    c1_2 = upsampling_block(c2_1, 32, 1, 2, [c1_1])
    c2_2 = upsampling_block(c3_1, 64, 2, 2, [c2_1])
    c3_2 = upsampling_block(c4_1, 128, 3, 2, [c3_1])
    c4_2 = upsampling_block(c5_1, 256, 4, 2, [c4_1])

    c1_3 = upsampling_block(c2_2, 32, 1, 3, [c1_1, c1_2])
    c2_3 = upsampling_block(c3_2, 64, 2, 3, [c2_1, c2_2])
    c3_3 = upsampling_block(c4_2, 128, 3, 3, [c3_1, c3_2])

    c1_4 = upsampling_block(c2_3, 32, 1, 4, [c1_1, c1_2, c1_3])
    c2_4 = upsampling_block(c3_3, 64, 2, 4, [c2_1, c2_2, c2_3])

    c1_5 = upsampling_block(c2_4, 32, 1, 5, [c1_1, c1_2, c1_3, c1_4])

    outputs = Conv2D(num_classes, (1, 1), padding='same', activation='softmax', name='output')(c1_5)

    model = Model(inputs=inputs, outputs=outputs, name='U-Net_plusplus')

    return model

unet_plusplus = build_unet_plusplus()

unet_plusplus.compile(
    loss = DiceLoss,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[fscore, mean_iou, 'accuracy']
)

try:
    unet_plusplus.fit(
        train_dataset,
        steps_per_epoch=train_steps,
        validation_data=val_dataset,
        validation_steps=valid_steps,
        batch_size=batch_size,
        epochs=epochs,
    )
except KeyboardInterrupt:
    print(f"{'-'*50}Training Stopped.{ '-'*50}")

Epoch 1/5


2025-08-05 09:02:48.149811: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-05 09:02:48.339885: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-05 09:02:48.903632: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-05 09:02:49.101813: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-05 09:02:56.401593: E external/local_xla/xla/stream_

288/288 ━━━━━━━━━━━━━━━━━━━━ 301s 726ms/step - accuracy: 0.2063 - f1-score: 0.0242 - iou_score: 0.0146 - loss: 0.9758 - val_accuracy: 0.0178 - val_f1-score: 0.0576 - val_iou_score: 0.0569 - val_loss: 0.9424
Epoch 2/5
288/288 ━━━━━━━━━━━━━━━━━━━━ 193s 666ms/step - accuracy: 0.0320 - f1-score: 0.0076 - iou_score: 0.0040 - loss: 0.9924 - val_accuracy: 0.0000e+00 - val_f1-score: 0.0145 - val_iou_score: 0.0145 - val_loss: 0.9855
Epoch 3/5
288/288 ━━━━━━━━━━━━━━━━━━━━ 204s 703ms/step - accuracy: 0.0015 - f1-score: 0.0015 - iou_score: 7.3175e-04 - loss: 0.9985 - val_accuracy: 0.0000e+00 - val_f1-score: 0.0145 - val_iou_score: 0.0145 - val_loss: 0.9855
Epoch 4/5
288/288 ━━━━━━━━━━━━━━━━━━━━ 192s 664ms/step - accuracy: 8.0526e-04 - f1-score: 0.0010 - iou_score: 5.2075e-04 - loss: 0.9990 - val_accuracy: 0.0000e+00 - val_f1-score: 0.0145 - val_iou_score: 0.0145 - val_loss: 0.9855
Epoch 5/5
288/288 ━━━━━━━━━━━━━━━━━━━━ 203s 701ms/step - accuracy: 0.0025 - f1-score: 0.0019 - iou_score: 9.3350e-04 -

In [16]:
linknet = sm.Linknet(
    backbone_name='resnet34', 
    encoder_weights='imagenet', 
    input_shape=shape, 
    classes=num_classes, 
    activation='softmax'
)

linknet.compile(
    loss = DiceLoss,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[fscore, mean_iou, 'accuracy']
)

try:
    linknet.fit(
        train_dataset,
        steps_per_epoch=train_steps,
        validation_data=val_dataset,
        validation_steps=valid_steps,
        batch_size=batch_size,
        epochs=epochs,
    )
except KeyboardInterrupt:
    print(f"{'-'*50}Training Stopped.{ '-'*50}")

85521592/85521592 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/5


2025-08-05 09:23:06.856693: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-05 09:23:07.046740: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-05 09:23:07.878472: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-05 09:23:08.071686: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-08-05 09:23:08.559738: E external/local_xla/xla/stream_

288/288 ━━━━━━━━━━━━━━━━━━━━ 189s 427ms/step - accuracy: 0.3137 - f1-score: 0.0571 - iou_score: 0.0373 - loss: 0.9429 - val_accuracy: 0.3741 - val_f1-score: 0.0302 - val_iou_score: 0.0216 - val_loss: 0.9698
Epoch 2/5
288/288 ━━━━━━━━━━━━━━━━━━━━ 120s 413ms/step - accuracy: 0.5065 - f1-score: 0.1323 - iou_score: 0.0931 - loss: 0.8677 - val_accuracy: 0.3747 - val_f1-score: 0.0361 - val_iou_score: 0.0262 - val_loss: 0.9639
Epoch 3/5
288/288 ━━━━━━━━━━━━━━━━━━━━ 119s 411ms/step - accuracy: 0.5303 - f1-score: 0.1529 - iou_score: 0.1096 - loss: 0.8471 - val_accuracy: 0.3809 - val_f1-score: 0.0317 - val_iou_score: 0.0224 - val_loss: 0.9683
Epoch 4/5
288/288 ━━━━━━━━━━━━━━━━━━━━ 198s 604ms/step - accuracy: 0.5696 - f1-score: 0.1737 - iou_score: 0.1266 - loss: 0.8263 - val_accuracy: 0.4777 - val_f1-score: 0.0848 - val_iou_score: 0.0610 - val_loss: 0.9152
Epoch 5/5
288/288 ━━━━━━━━━━━━━━━━━━━━ 121s 415ms/step - accuracy: 0.5912 - f1-score: 0.1845 - iou_score: 0.1359 - loss: 0.8155 - val_accuracy

In [ ]:
def convolution_block(
    input,
    name,
    num_filters=256,
    kernel=3,
    dilation_rate=1,
    use_bias=False,
):
    x = Conv2D(
        num_filters,
        kernel_size=kernel,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        name=f"{name}_conv_{num_filters}_{kernel}_{dilation_rate}"
    )(input)
    x = BatchNormalization(name=f"{name}_bn_{num_filters}_{kernel}_{dilation_rate}")(x)
    x = Activation("relu", name=f"{name}_relu_{num_filters}_{kernel}_{dilation_rate}")(x)

    return x

def AtrousSpatialPyramidPooling(input):

    dim = input.shape

    x = AveragePooling2D(
        pool_size=(dim[-3], dim[-2]),
        name="global_avg_pool"
    )(input)
    x = convolution_block(x, kernel=1, use_bias=True, name="aspp_conv1")
    out_pool = UpSampling2D(
        size=(dim[-3] // x.shape[1], dim[-2] // x.shape[2]),
        interpolation="bilinear",
        name="aspp_upsample"
    )(x)

    out_1 = convolution_block(input, kernel=1, dilation_rate=1, name="aspp_conv_1x1")
    out_6 = convolution_block(input, kernel=3, dilation_rate=6, name="aspp_conv_3x3_d6")
    out_12 = convolution_block(input, kernel=3, dilation_rate=12, name="aspp_conv_3x3_d12")
    out_18 = convolution_block(input, kernel=3, dilation_rate=18, name="aspp_conv_3x3_d18")

    x = Concatenate(axis=-1, name="aspp_concat")([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel=1, name="aspp_project")
    return output


def DeeplabV3Plus(shape=shape, num_classes=num_classes, backbone='resnet50'):

    input = Input(shape=shape, name='input_image')

    if backbone == 'resnet50':
        resnet50 = keras.applications.ResNet50(
            weights="imagenet", include_top=False, input_tensor=input
        )
        feature_1 = resnet50.get_layer('conv4_block6_2_relu').output
        feature_2 = resnet50.get_layer('conv2_block3_2_relu').output

    input_1 = AtrousSpatialPyramidPooling(feature_1)
    input_1 = UpSampling2D(
        size=(shape[0] // 4 // input_1.shape[1], shape[1] // 4 // input_1.shape[2]),
        interpolation="bilinear",
        name="aspp_upsample_final"
    )(input_1)

    input_2 = convolution_block(feature_2, num_filters=48, kernel=1, name="low_level_feature")

    x = concatenate([input_1, input_2], axis=-1, name="decoder_concat")
    x = convolution_block(x, name="decoder_conv1")
    x = convolution_block(x, name="decoder_conv2")
    x = UpSampling2D(
        size=(shape[0] // x.shape[1], shape[1] // x.shape[2]),
        interpolation="bilinear",
        name="final_upsample"
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same", name="output_layer")(x)
    return keras.Model(inputs=input, outputs=model_output, name="DeepLabv3plus")


deeplabv3 = DeeplabV3Plus()

deeplabv3.compile(
    loss = DiceLoss,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[fscore, mean_iou, 'accuracy']
)

try:
    deeplabv3.fit(
        train_dataset,
        steps_per_epoch=train_steps,
        validation_data=val_dataset,
        validation_steps=valid_steps,
        batch_size=batch_size,
        epochs=epochs,
    )
except KeyboardInterrupt:
    print(f"{'-'*50}Training Stopped.{ '-'*50}")

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 2387s 33s/step - accuracy: 0.0866 - f1-score: 0.0135 - iou_score: -0.0022 - loss: 0.9865 - val_accuracy: 0.3467 - val_f1-score: 0.0815 - val_iou_score: 0.0570 - val_loss: 0.9185
Epoch 2/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 2354s 33s/step - accuracy: 0.0667 - f1-score: 0.0885 - iou_score: 0.0682 - loss: 0.9115 - val_accuracy: 0.0226 - val_f1-score: 0.0571 - val_iou_score: 0.0454 - val_loss: 0.9429
Epoch 3/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 2354s 33s/step - accuracy: 0.3357 - f1-score: 0.1760 - iou_score: 0.0197 - loss: 0.8240 - val_accuracy: 0.3467 - val_f1-score: 0.0831 - val_iou_score: 0.0575 - val_loss: 0.9169
Epoch 4/5
 2/72 ━━━━━━━━━━━━━━━━━━━━ 35:09 30s/step - accuracy: 0.2719 - f1-score: 0.0855 - iou_score: 0.0613 - loss: 0.9145

In [16]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    
    train_dataset = prepare_dataset(img_train, mask_train, model_type='standard')
    val_dataset = prepare_dataset(img_val, mask_val, model_type='standard')

    print(f"Dataset prepared.\n{'-'*100}\n\nMODEL TRAINING\n\n{'-'*100}\n\n")

    model = DeeplabV3Plus()
    
    model.compile(
        loss = dice_loss,
        optimizer=tf.keras.optimizers.Adam(lr),
        metrics=[fscore, mean_iou, 'accuracy']
    )


    try:
        model.fit(
            train_dataset,
            steps_per_epoch=train_steps,
            validation_data=val_dataset,
            validation_steps=valid_steps,
            callbacks=callbacks,
            batch_size=batch_size,
            epochs=epochs,
        )
    except KeyboardInterrupt:
        print(f"\n\n{'-'*70}\nTraining Stopped.\n{ '-'*70}")

Dataset prepared.
----------------------------------------------------------------------------------------------------

MODEL TRAINING

----------------------------------------------------------------------------------------------------


Epoch 1/200


I0000 00:00:1754412259.301215    1560 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1754412259.732963    1559 cuda_dnn.cc:529] Loaded cuDNN version 90300


300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 894ms/step - accuracy: nan - f1-score: nan - iou_score: nan - loss: nan
Epoch 1: val_loss improved from inf to 1.98599, saving model to /kaggle/working/models/model.keras
300/300 ━━━━━━━━━━━━━━━━━━━━ 365s 1s/step - accuracy: nan - f1-score: nan - iou_score: nan - loss: nan - val_accuracy: 0.0668 - val_f1-score: 0.0706 - val_iou_score: 0.0450 - val_loss: 1.9860 - learning_rate: 1.0000e-04
Epoch 2/200
197/300 ━━━━━━━━━━━━━━━━━━━━ 1:31 892ms/step - accuracy: nan - f1-score: nan - iou_score: nan - loss: nan

----------------------------------------------------------------------
Training Stopped.
----------------------------------------------------------------------


Epoch 1/200
I0000 00:00:1754410505.617745     455 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1754410505.661842     456 cuda_dnn.cc:529] Loaded cuDNN version 90300
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0976 - f1-score: 0.0889 - iou_score: 0.3880 - loss: 1.8221
Epoch 1: val_loss improved from inf to 1.83158, saving model to /kaggle/working/models/model.keras
75/75 ━━━━━━━━━━━━━━━━━━━━ 194s 2s/step - accuracy: 0.0976 - f1-score: 0.0889 - iou_score: 0.3880 - loss: 1.8221 - val_accuracy: 0.0251 - val_f1-score: 0.0842 - val_iou_score: 0.1372 - val_loss: 1.8316 - learning_rate: 0.0010
Epoch 2/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0768 - f1-score: 0.0879 - iou_score: 0.0186 - loss: 1.8241
Epoch 2: val_loss improved from 1.83158 to 1.81453, saving model to /kaggle/working/models/model.keras
75/75 ━━━━━━━━━━━━━━━━━━━━ 138s 2s/step - accuracy: 0.0768 - f1-score: 0.0879 - iou_score: 0.0185 - loss: 1.8241 - val_accuracy: 0.0187 - val_f1-score: 0.0927 - val_iou_score: 0.0843 - val_loss: 1.8145 - learning_rate: 0.0010
Epoch 3/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2145 - f1-score: 0.0745 - iou_score: 0.0503 - loss: 1.8510
Epoch 3: val_loss did not improve from 1.81453
75/75 ━━━━━━━━━━━━━━━━━━━━ 150s 2s/step - accuracy: 0.2145 - f1-score: 0.0745 - iou_score: 0.0503 - loss: 1.8510 - val_accuracy: 0.0430 - val_f1-score: 0.0588 - val_iou_score: 0.0344 - val_loss: 1.8824 - learning_rate: 0.0010
Epoch 4/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2948 - f1-score: 0.1014 - iou_score: 0.0853 - loss: 1.7972
Epoch 4: val_loss did not improve from 1.81453
75/75 ━━━━━━━━━━━━━━━━━━━━ 134s 2s/step - accuracy: 0.2948 - f1-score: 0.1014 - iou_score: 0.0853 - loss: 1.7972 - val_accuracy: 0.0374 - val_f1-score: 0.0853 - val_iou_score: 1.3990 - val_loss: 1.8295 - learning_rate: 0.0010
Epoch 5/200
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2835 - f1-score: 0.0987 - iou_score: 0.0534 - loss: 1.8025
Epoch 5: val_loss did not improve from 1.81453
75/75 ━━━━━━━━━━━━━━━━━━━━ 126s 2s/step - accuracy: 0.2835 - f1-score: 0.0987 - iou_score: 0.0534 - loss: 1.8025 - val_accuracy: 0.0091 - val_f1-score: 0.0833 - val_iou_score: 0.0594 - val_loss: 1.8334 - learning_rate: 0.0010
Epoch 6/200
59/75 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.1897 - f1-score: 0.1326 - iou_score: 0.0394 - loss: 1.7347

----------------------------------------------------------------------
Training Stopped.
----------------------------------------------------------------------

In [22]:
! sudo fuser -v /dev/nvidia*

                     USER        PID ACCESS COMMAND
/dev/nvidia0:        root         38 F.... nvidia-smi
                     root        422 F...m python3
/dev/nvidia1:        root         38 F.... nvidia-smi
                     root        422 F...m python3
/dev/nvidiactl:      root         38 F.... nvidia-smi
                     root        422 F...m python3
/dev/nvidia-uvm:     root         38 F.... nvidia-smi
                     root        422 F...m python3


In [ ]:
! sudo kill -9 422
! sudo kill -9 38

In [ ]:
! rm -rf /kaggle/working/logs

In [ ]:
! rm -rf /kaggle/working/models

In [17]:
print(tf.config.list_logical_devices('TPU'))

[LogicalDevice(name='/device:TPU:0', device_type='TPU'), LogicalDevice(name='/device:TPU:1', device_type='TPU'), LogicalDevice(name='/device:TPU:2', device_type='TPU'), LogicalDevice(name='/device:TPU:3', device_type='TPU'), LogicalDevice(name='/device:TPU:4', device_type='TPU'), LogicalDevice(name='/device:TPU:5', device_type='TPU'), LogicalDevice(name='/device:TPU:6', device_type='TPU'), LogicalDevice(name='/device:TPU:7', device_type='TPU')]


In [22]:
print("Number of replicas:", strategy.num_replicas_in_sync)

Number of replicas: 1
